<a href="https://colab.research.google.com/github/khorzhengyu/project518/blob/main/Features_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#modules for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.ticker as ticker

#modules for model building
#algorithms for sampling
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

#modules for machine learning algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import collections
from sklearn import datasets, metrics, preprocessing


# modules for evaluation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report,confusion_matrix
from xgboost import XGBClassifier
from xgboost import Booster
from xgboost import DMatrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# read the csv file
df_train= pd.read_csv('gdrive/My Drive/fraudTrain.csv')
df_test = pd.read_csv('gdrive/My Drive/fraudTest.csv')

In [ ]:
#concatenating the two datasets together
df = pd.concat([df_train,df_test], ignore_index =True)

In [ ]:
# removing column names that contain '_'
df.columns = df.columns.str.replace('_', '')

In [ ]:
# taking 90000 data for computation
df = df.sample(frac=1, random_state=2)
df = df.head(n=90000)
df.isfraud.value_counts()

In [ ]:
# Adding a date variable in the format: YYYY-MM-DD
df['transdatetranstime'] = pd.to_datetime(df['transdatetranstime'])
df['transdate'] = df['transdatetranstime'].dt.date
df['transdate'] = pd.to_datetime(df['transdate'])

In [ ]:
#sort by transaction date
df.sort_values(by = 'transdate', inplace = True)

In [ ]:
# Adding 'recnum' as index
df['recnum'] = range(1, len(df) + 1)

In [ ]:
#remove the first unused column
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
# Creating candidates variables ####

from copy import deepcopy

# Make a copy of DataFrame to avoid modifying the original data
df_var = deepcopy(df)

# Convert values to string type for selected columns
cols_convert = ['ccnum', 'merchlat','merchlong']
for item in cols_convert:
    df_var[item] = df_var[item].astype(str)

# Print data types of the columns in the modified DataFrame
print(df_var.dtypes)

In [ ]:
# Make variable combos
df_var['card-merchant'] = df_var['ccnum'] + df_var['merchant']
#df_var['card-merchant-location'] = df_var['cc_num'] + df_var['merch_lat'] + df_var['merch_long']

In [ ]:
df_var['ccnum'] = df_var['ccnum'].astype(int)
df_var['merchlat'] = df_var['merchlat'].astype(float)
df_var['merchlong'] = df_var['merchlong'].astype(float)

In [ ]:
## Day-since variable
# Creating the function for the variable
def ds(dataframe, g1, g2, name):
#     'Helps with calculating the day since variables'
    day_since = dataframe.groupby(g1)[g1].first()
    day_since = day_since.rename_axis(['None' for i in range(len(g1))]).groupby(g2).diff()
    day_since.columns = [name]
    day_since = day_since.rename_axis(g1)
    day_since[name] = day_since[name].dt.days.fillna(0)
    day_since = day_since.reset_index()
    return day_since

In [ ]:
# Creating Day-Since Variable
start_daySince = pd.datetime.now()
day_card = ds(df_var, ['ccnum', 'transdate'], 'ccnum', 'card_daysSince')
print("done!", pd.datetime.now()-start_daySince)
day_card

In [ ]:
# day_card

In [ ]:
from datetime import datetime
time_ds_all=datetime.now()
# Calculate the Days Since variables for the required columns
ds_cols = ['ccnum','merchant','card-merchant']

ds_dict={}
for col in ds_cols:
    curr_time=datetime.now()
    curr_name = 'daysSince_' + col

    # Calculate the days-since variable (ds) and assign it to a global variable (curr_name)
    vars()[curr_name] = ds(df_var, [col, 'transdate'], col, col+'_daysSince')
    ds_dict[curr_name] = vars()[curr_name] # Save results to a dictionary

    print("Done with:", col, "; Time:", datetime.now()-curr_time)

print("DONE!", datetime.now()-time_ds_all)

In [ ]:
ds_dict.keys()
print(type(ds_dict.keys()))

In [ ]:
# Create a copy of the main DataFrame to avoid modifying the original data
df_ds = df_var.copy()

# Merge the days-since variables with the main dataset
for item in ds_dict.keys():
    col_variable = item.split('_')[1]
    df_ds = pd.merge(df_ds, vars()[item], how='left', left_on=[col_variable,'transdate'], right_on=[col_variable,'transdate'])

In [ ]:
df_ds.to_csv('df_daysSince.csv')

In [ ]:
from datetime import datetime
# Create Columns for the Necessary Time Periods ##
#This makes new columns for the various time periods.
# Make a list of variable combinations to iterate through and create time-related variables
var_combos = ['ccnum','merchant','card-merchant']

# Create column names
time_list = [0,1,3,7,14,30]
time_joined =[]
for time in time_list:
    time_joined.append('join_ts_'+str(time))

start_copy = datetime.now()
df_var1 = deepcopy(df_var)
df_var2 = deepcopy(df_var)
print('copy time', datetime.now()-start_copy)

# Creating columns for time
start_loop=datetime.now()
for time in time_list:
    temp_endTime = 'join_ts_' + str(time)
    df_var2[temp_endTime] = df_var2['transdate'] + dt.timedelta(days = time)
print('first loop', datetime.now()-start_loop)

In [ ]:
# Create the Frequency Candidate Variables ##
start_loop2=datetime.now()
df_final = deepcopy(df_var.set_index('recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum']].copy()
    df_var4.rename(columns={'recnum':'recnum2'},inplace=True) # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts_' + str(time)
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]
        temp_groupby = df2_temp[['recnum','transdate']].groupby('recnum')
        temp_name = item + '_' + 'freq' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'count')().add_prefix(temp_name), left_index=True, right_index=True, how='left')

print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
df_final.loc[df_final['ccnum'] == 6011504998544485]

In [ ]:
start_loop3= datetime.now()
groupbyvar_denom = deepcopy(var_combos)
days_numer = ['0','1']
days_denom = ['7','14','30']

for b in groupbyvar_denom:
    for c in days_numer:
        for d in days_denom:
            temp = d
            df_final[b + '_' + c + '_dayfreq' + '_div_' + d + '_dayfreq' + '_velchange'] = \
                df_final[b + '_freq' + c + '_transdate'] / \
                df_final[b + '_freq' + d + '_transdate'] / float(temp)
print('third loop', datetime.now() - start_loop3)

In [ ]:
# Reset the index so that the Recnum returns to being a normal column
df_final.reset_index(inplace=True)

In [ ]:
# Merge df_final (frequency and velocity change) and df_ds (days-since)
df_all_vars = pd.merge(df_final, df_ds, on=list(df_var.columns))

In [ ]:
from typing import List
# Optimize Function
def optimize_floats(df: pd.DataFrame) -> pd.DataFrame:
    floats = df.select_dtypes(include=['float64']).columns.tolist()
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
    return df


def optimize_ints(df: pd.DataFrame) -> pd.DataFrame:
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    return df


def optimize_objects(df: pd.DataFrame, datetime_features: List[str]) -> pd.DataFrame:
    for col in df.select_dtypes(include=['object']):
        if col not in datetime_features:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df



def optimize(df: pd.DataFrame, datetime_features: List[str] = []):
    return optimize_floats(optimize_ints(optimize_objects(df, datetime_features)))

In [ ]:
# Make variable combos
main_dataset_filled = deepcopy(df)
main_dataset_filled['ccnum_|_merchant'] = main_dataset_filled['ccnum'].astype('str') + main_dataset_filled['merchant'].astype('str')
 # main_dataset_filled['ccnum_|_merchantlocation'] = main_dataset_filled['cc_num'].astype('str') + main_dataset_filled['merch_lat'].astype('str') + main_dataset_filled['merch_long'].astype('str')

In [ ]:
var_combos = ['ccnum','merchant','ccnum_|_merchant']

In [ ]:
time_joined =['join_ts1']
for num in time_list:
    time_joined.append('join_ts2_'+str(num))

In [ ]:
# Making time variables
start=datetime.now()

time_list = [0,1,3,7,14,30]
main_dataset_filled['join_ts1']=main_dataset_filled['transdate']
# dt_i
for dt_i in time_list:
    time = 'join_ts2_'+str(dt_i)
    main_dataset_filled[time]=main_dataset_filled['transdate'] + dt.timedelta(dt_i)
print('Completed ',datetime.now()-start)

In [ ]:
df_var1 = main_dataset_filled.copy()
df_var2 = main_dataset_filled.copy()

In [ ]:
# Total Amount Variables
start_loop2= datetime.now()
df_final = deepcopy(main_dataset_filled.set_index('recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True) # this causes a warning to arise
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'totalamount' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'sum')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
# Median amount
start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'median' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'median')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
# Mean Variables

start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'mean' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'mean')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
 # Max Variables
start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'max' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'max')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
df_final = optimize(df_final,['transdate'])

In [ ]:
all_columns = df_final.columns.tolist()
all_columns

In [ ]:
start_column = df_final.columns.get_loc('ccnum_totalamount0_amt2')
for col_names in df_final.columns[start_column:]:
#  the number of days in each column is the column index mod length of time array
    number_index = time_list[(df_final.columns.get_loc(col_names)%len(time_list))-1]
#     we split the name of the array to keep the first part and use it to create the new column names
    new_col_name = col_names.split(str(number_index))[0]+'_'+str(number_index)+'_actual'
#     print(new_col_name)
    df_final[new_col_name] = df_final["amt"]/df_final[col_names]

In [ ]:
# Amount Velocity Change Variables
# iterates through  the variables created with the var_combos columns and finds the relative
# velocity variables. Divides the number of days of days_numer array with the days_denom one
start_loop3= datetime.now()
groupbyvar_denom = deepcopy(var_combos)
days_numer = ['0','1']
days_denom = ['7','14','30']
# ccnum_totalamount0_amt2_/_ccnum_totalamount3_amt2
for b in groupbyvar_denom:
    for c in days_numer:
        for d in days_denom:
            temp = d
            df_final[b + '_' + c + '_dayamount' + '_div_' + d + '_dayamount' + '_velchange'] = \
                df_final[b+'_totalamount'+c+'_amt2'] / \
                df_final[b+'_totalamount'+d+'_amt2'] / float(temp)
print('third loop', datetime.now() - start_loop3)

In [ ]:
df_all_vars = optimize(df_all_vars)

In [ ]:
# df_all_vars contains frequency variables, velocity change variables, and days since variables
# merge1_df contains amount variables, velocity change amount variables, and benford's law variables
df_all_vars_final = pd.merge(df_all_vars, df_final, how='left',left_on=df.columns.to_list(),right_on=df.columns.to_list())

In [ ]:
df_all_vars_final = df_all_vars_final.drop(columns=['card-merchant', 'join_ts1', 'join_ts2_0',
       'join_ts2_1', 'join_ts2_3', 'join_ts2_7', 'join_ts2_14',
       'join_ts2_30'])
df_all_vars_final.head()

In [ ]:
#### Examples ########
#
#
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['ccnum', 'transdate', 'amt', 'ccnum_freq0_transdate', 'ccnum_freq1_transdate' ,'ccnum_freq3_transdate', 'ccnum_freq7_transdate']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 6011504998544485
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

# Define the range of indices you want to show
start_index = 5
end_index = 200

# Filter rows within the specified index range
filtered_df = filtered_df.iloc[start_index:end_index+1]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'merchant', 'transdate', 'amt', 'ccnum_totalamount0_amt2', 'ccnum_|_merchant_totalamount7_amt2' ,'ccnum_|_merchant_totalamount_7_actual']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'transdate', 'amt', 'ccnum_totalamount0_amt2', 'ccnum_totalamount7_amt2' ,'ccnum_0_dayamount_div_7_dayamount_velchange']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'transdate', 'amt', 'ccnum_freq0_transdate', 'ccnum_freq7_transdate' ,'ccnum_0_dayfreq_div_7_dayfreq_velchange']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [ ]:
df = df_all_vars_final.copy()
df.head()